In [6]:
import json
import asyncio

from aiohttp import ClientSession, BasicAuth
from python_opensky import OpenSky, BoundingBox
from pyensign.ensign import authenticate, publisher

# Configuration

In [7]:
OPENSKY_CREDS_PATH="opensky-creds.json"
ENSIGN_CREDS_PATH="ensign-creds.json"

def load_opensky_creds():
    with open(OPENSKY_CREDS_PATH) as f:
        creds = json.load(f)
        return BasicAuth(login=creds["username"], password=creds["password"])

OPENSKY_CREDS = load_opensky_creds()

In [8]:
# Continental US
# TODO : update for continental US - (19, 65, -162, -67)
BOUNDING_BOX = BoundingBox(
    min_latitude=30,
    max_latitude=40,
    min_longitude=-120,
    max_longitude=-90,
)

INTERVAL_SEC=10

# Publish Flight Updates

In [9]:
@publisher("flight-positions")
async def get_flights():
    async with ClientSession() as session:
        async with OpenSky(session=session) as opensky:
            await opensky.authenticate(OPENSKY_CREDS)
            
            response = await opensky.get_states(bounding_box=BOUNDING_BOX)
            for vector in response.states:
                yield {
                    "icao24": vector.icao24,
                    "callsign": vector.callsign,
                    "origin_country": vector.origin_country,
                    "time_position": vector.time_position,
                    "last_contact": vector.last_contact,
                    "longitude": vector.longitude,
                    "latitude": vector.latitude,
                    "geo_altitude": vector.geo_altitude,
                    "on_ground": vector.on_ground,
                    "velocity": vector.velocity,
                    "true_track": vector.true_track,
                    "vertical_rate": vector.vertical_rate,
                    "sensors": vector.sensors,
                    "barometric_altitude": vector.barometric_altitude,
                    "transponder_code": vector.transponder_code,
                    "special_purpose_indicator": vector.special_purpose_indicator,
                    "position_source": vector.position_source,
                    "category": vector.category,
                }
                
@authenticate(cred_path=ENSIGN_CREDS_PATH)
async def poll_flights():
    for i in range(10):
        async for data in get_flights():
            print(data)
        await asyncio.sleep(INTERVAL_SEC)
            
    # Wait for pending events to get published
    await asyncio.sleep(1)

In [12]:
await poll_flights()

{'icao24': 'a1befc', 'callsign': 'UAL2096 ', 'origin_country': 'United States', 'time_position': 1697525043, 'last_contact': 1697525043, 'longitude': -115.4701, 'latitude': 35.7204, 'geo_altitude': 11148.06, 'on_ground': False, 'velocity': 256.55, 'true_track': 48.41, 'vertical_rate': 0.0, 'sensors': None, 'barometric_altitude': 10668.0, 'transponder_code': '6727', 'special_purpose_indicator': False, 'position_source': <PositionSource.ADSB: 0>, 'category': <AircraftCategory.NO_ADS_INFORMATION: 1>}
{'icao24': 'ab034d', 'callsign': 'AAL9794 ', 'origin_country': 'United States', 'time_position': 1697525042, 'last_contact': 1697525042, 'longitude': -117.4445, 'latitude': 34.3702, 'geo_altitude': 12108.18, 'on_ground': False, 'velocity': 243.59, 'true_track': 259.78, 'vertical_rate': 0.0, 'sensors': None, 'barometric_altitude': 11582.4, 'transponder_code': None, 'special_purpose_indicator': False, 'position_source': <PositionSource.ADSB: 0>, 'category': <AircraftCategory.HEAVY: 6>}
{'icao24

OpenSkyConnectionError: Timeout occurred while connecting to the OpenSky API